# Data Preparation

In this notebook, we will prepare the data for training and testing.
Since we are using two datasets (1M and 32M), we will prepare both datasets separately.
We will be using functions from the utils folder to help with data splitting and preparation and the DataFrames created in the previous notebook.

## 0. Utils folder path setup for imports

Before we start we need to add the utils folder to the system path so that we can import the functions defined there.

In [3]:
import sys
from pathlib import Path as pth
import gc

# Add the utils folder to the system path
project_root = pth.cwd().parent # Should go from ML_Project/notebooks to ML_Project

sys.path.insert(0,str(project_root))

print("Utils folder added to system path for imports.")

Utils folder added to system path for imports.


## 1. Importing Necessary Libraries and Functions

This cell imports all the necessary libraries and functions for data preparation.

In [4]:
import pandas as pd
from utils.preprocessing import format_rating_data, format_movies_data, load_surprise_dataset, split_surprise_dataset, load_movies_data, load_rating_data, merge_datasets